# BM25 (Best Matching 25) - Keyword Search

✅ Tip for Testing BM25
To test BM25:

- Use a tokenizer (simple whitespace or better: regex-based)

- Clean/normalize (lowercase, optional stemming/stopword removal)

- Rank documents based on BM25 scores given a query

$$
IDF(q_i) = \log \left( \frac{N - n(q_i) + 0.5}{n(q_i) + 0.5} + 1 \right)
$$


In [60]:
import math

def compute_probabilistic_idf(term, docs):
    '''Arguments
        term - it should be clean or normalize
        docs - it should be clean or normalize and tokonize
    '''
    doc_count = sum(term in doc for doc in docs)
    numerator = len(docs) - doc_count + 0.5
    denominator = doc_count + 0.5
    log_arg = (numerator / denominator) + 1
    return math.log(log_arg)

$$
\text{BM25}(q, D) = \sum_{i=1}^{n} IDF(q_i) \cdot \frac{f(q_i, D) \cdot (k_1 + 1)}{f(q_i, D) + k_1 \cdot \left(1 - b + b \cdot \frac{|D|}{\text{avgdl}}\right)}
$$


In [77]:
def compute_tf(term, document):
    words = document.split()
    term_count = words.count(term)
    return term_count / len(words) if words else 0

def compute_avgdl(documents):
    total_document_length = sum(len(doc.split()) for doc in documents)
    return total_document_length / len(documents)

def compute_bm25(query, document, documents, k1=1.5, b=0.75):
    bm25_score = 0
    
    doc_len = len(document.split())
    avgdl = compute_avgdl(documents)
    for term in query.split():
        tf = compute_tf(term, document)
        idf = compute_probabilistic_idf(term, documents)
        numerator = tf * (k1 + 1)
        doc_norm = (1 - b) + (b * (doc_len / avgdl))
        denominator = tf + (k1 * doc_norm)
        score = idf * (numerator / denominator)
        bm25_score += score
    
    return bm25_score
        

## Sample Docs and Queries

In [62]:
documents = [
    "Paano mag-apply ng Japan tourist visa sa Pilipinas?",
    "Ano ang mga requirements sa student visa ng Japan?",
    "Ilang araw ang processing time ng Japan visa application?",
    "Saan ang location ng Japan embassy sa Manila?",
    "Ano ang mga dokumentong kailangan para sa Japan visa?",
    "Puwede bang mag walk-in sa Japan embassy?",
    "Paano i-track ang status ng aking visa application?",
    "Kailangan ba ng show money para sa Japan tourist visa?",
    "Puwede bang mag-apply ng multiple entry visa papuntang Japan?",
    "Sino ang pwedeng kontakin para sa tulong sa visa processing?"
]

queries = [
    "requirements for Japan visa",
    "how to apply for tourist visa",
    "Japan embassy location",
    "can I track my visa",
    "do I need show money for Japan"
]


## Preprocessing

In [63]:
english_stopwords = [
    "a", "an", "the", "and", "or", "but", "if", "while", "at", "by", "for", "from",
    "in", "into", "on", "onto", "of", "to", "with", "without", "is", "am", "are", 
    "was", "were", "be", "been", "being", "do", "does", "did", "doing", "have", 
    "has", "had", "having", "will", "would", "shall", "should", "can", "could", 
    "may", "might", "must", "this", "that", "these", "those", "i", "you", "he", 
    "she", "it", "we", "they", "me", "him", "her", "us", "them", "my", "your", 
    "his", "her", "its", "our", "their", "mine", "yours", "ours", "theirs"
]

filipino_stopwords = [
    "ako", "ikaw", "siya", "kami", "kayo", "sila", "ko", "mo", "niya", "namin", "ninyo", "nila",
    "ang", "ng", "sa", "mga", "kay", "para", "may", "wala", "ito", "iyan", "iyon", "dito", 
    "diyan", "doon", "kaya", "pero", "at", "o", "dahil", "kung", "kapag", "habang", "bakit", 
    "paano", "saan", "kailan", "lahat", "lamang", "rin", "din", "pa", "na", "nang", "ngunit"
]


In [64]:
def tokenize(document):
    return document.split()

def lowercase(document):
    return document.lower()

def remove_stopwords(document, stopwords=english_stopwords+filipino_stopwords):
    candidate_words = [word for word in document.split() if word not in stopwords]
    return " ".join(candidate_words)

def preprocess(document):
    lowercase_doc = lowercase(document)
    return remove_stopwords(lowercase_doc)

test_document = "hello word of and ang mga"

remove_stopwords(test_document)

'hello word'

## Probabilistic IDF

In [65]:
clean_docs = [lowercase(doc) for doc in documents]
preprocessed_docs = [tokenize(doc) for doc in clean_docs]
preprocessed_docs

query = queries[0]
clean_query = remove_stopwords(lowercase(query))
preprocessed_query = tokenize(clean_query)
query_i = preprocessed_query[0]

print(f'query: {query}')
print(f'clean_query: {clean_query}')
compute_probabilistic_idf(query_i, preprocessed_docs) # Computes how much Rare the Word is

query: requirements for Japan visa
clean_query: requirements japan visa


1.992430164690206

### Extract Top IDF Words Only

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer

processed_docs = [remove_stopwords(doc) for doc in documents]

# Use only IDF (ignore TF by using binary=True)
vectorizer = TfidfVectorizer(binary=True, use_idf=True, norm=None)
vectorizer.fit(documents)

idf_scores = dict(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))
sorted_idf = sorted(idf_scores.items(), key=lambda x: x[1], reverse=True)

# Show top informative terms (rare terms)
for word, idf in sorted_idf[:10]:
    print(f"{word}: IDF = {idf:.2f}")


aking: IDF = 2.70
araw: IDF = 2.70
ba: IDF = 2.70
dokumentong: IDF = 2.70
entry: IDF = 2.70
ilang: IDF = 2.70
in: IDF = 2.70
kontakin: IDF = 2.70
location: IDF = 2.70
manila: IDF = 2.70


## BM25 Simulation

In [78]:
preprocessed_docs = [preprocess(document) for document in documents]
print(preprocessed_docs)
preprocessed_queries = [preprocess(query) for query in queries]
print(preprocessed_queries)

query = preprocessed_queries[0]
print(query)

for idx, doc in enumerate(preprocessed_docs):
    score = compute_bm25(query, doc, preprocessed_docs)
    print(f"Doc {idx+1}: BM25 score = {score:.4f}")

['how apply japan visa', 'visa requirements filipino tourists', 'japan embassy location hours', 'how long visa processing take', 'track visa application']
['requirements japan visa', 'how apply tourist visa', 'japan embassy location', 'track visa', 'need show money japan']
requirements japan visa
Doc 1: BM25 score = 0.4154
Doc 2: BM25 score = 0.5978
Doc 3: BM25 score = 0.3127
Doc 4: BM25 score = 0.0726
Doc 5: BM25 score = 0.1545


# Getting Top 3 Documents Simulation using BM25

In [81]:
def get_top3_docs(query, documents):
    preprocessed_docs = [preprocess(document) for document in documents]
    preprocessed_query = preprocess(query)

    scores = []
    for idx, doc in enumerate(preprocessed_docs):
        score = compute_bm25(preprocessed_query, doc, documents)
        scores.append((idx, score))

    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    top3_docs = [
        documents[sorted_scores[0][0]],
        documents[sorted_scores[1][0]],
        documents[sorted_scores[2][0]],
    ]
    return top3_docs

In [83]:
query = "requirements"

get_top3_docs(query, documents)

['visa requirements for filipino tourists',
 'how to apply for japan visa',
 'japan embassy location and hours']